In [1]:
import findspark
findspark.init()


In [2]:
import findspark
findspark.init()

import os
print(os.environ.get("JAVA_HOME"))


C:\Program Files\Java\jdk-11


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when, round
from functools import reduce
import operator


spark = SparkSession.builder \
    .appName("Test PySpark in Jupyter") \
    .getOrCreate()


In [ ]:
print("All Students : ")
df = spark.read.csv("students.csv", header=True, inferSchema=True)
df.toPandas()


All Student : 


,Roll No,Candidate Name,EM-II,PPS,CHEM/PHY,EG/MECH,BXE/BEE
0,F101,RAI ADITYA RAJKUMAR,31,34,26,42,22
1,F102,DHOKCHAULE SHREYA RAVINDRA,35,39,25,54,21
2,F103,GORE MANOJ TUKARAM,46,45,26,58,19
3,F104,GAVALI ROHIT PUNJARAM,19,18,18,39,7
4,F105,GAIKWAD AMIT ANKUSH,22,24,29,52,18
...,...,...,...,...,...,...,...
489,F858,DHOMSE TUSHAR VILAS,12,2,12,20,4
490,F859,DAUND PIYUSH ASHOK,AB,AB,AB,AB,AB
491,F860,KARAD SHRUTI BABASAHEB,14,14,30,30,5
492,F861,BARGAJE ADITYA GOVIND,10,10,15,20,3


In [5]:
print(df.columns)

['Roll No', 'Candidate Name', 'EM-II', 'PPS', 'CHEM/PHY', 'EG/MECH', 'BXE/BEE']


In [6]:
print("Absent Student in one or more Subject : ")
ab = df.filter(
    (col("EM-II") == "AB") |
    (col("PPS") == "AB") |
    (col("CHEM/PHY") == "AB") |
    (col("EG/MECH") == "AB") |
    (col("BXE/BEE") == "AB")
)
ab.show()


Absent Student in one or more Subject : 
+-------+--------------------+-----+---+--------+-------+-------+
|Roll No|      Candidate Name|EM-II|PPS|CHEM/PHY|EG/MECH|BXE/BEE|
+-------+--------------------+-----+---+--------+-------+-------+
|   F107|VINCHURKAR PARTH ...|   AB| AB|      29|     19|      5|
|   F114| CHAVAN VAIBHAV AJIT|   AB| AB|       0|     AB|     AB|
|   F121| BATTISE MEET ASHISH|   AB| AB|       0|     AB|     AB|
|   F129|NANDURGE KALYANI ...|   25| 24|      40|     12|     AB|
|   F132|PATIL BHAGYASHREE...|   18| 19|      37|     AB|      0|
|   F133|KOLHE RUTUJA RAJE...|   AB| AB|       0|     AB|     AB|
|   F137|  YADAV OMKAR SUDHIR|   10| AB|       0|     24|      3|
|   F138| MORE JAYESH JAGDISH|   10| 12|      26|      0|     AB|
|   F144|  KALE KARAN SHYAMAL|   AB| AB|       0|     AB|     AB|
|   F154| ARYAN KEVALDAS TURE|    5|  7|       9|     AB|      0|
|   F155| GAIKWAD YASH GANESH|   AB| AB|       0|     AB|     AB|
|   F201|THORAT AISHWARYA ...|    3

In [7]:
print("Absent Student in all subjects : ")
AB_1 = df.filter(
    (col("EM-II") == "AB") &
    (col("PPS") == "AB") &
    (col("CHEM/PHY") == "AB") &
    (col("EG/MECH") == "AB") &
    (col("BXE/BEE") == "AB")
)
AB_1.show()

Absent Student in all subjects : 
+-------+--------------------+-----+---+--------+-------+-------+
|Roll No|      Candidate Name|EM-II|PPS|CHEM/PHY|EG/MECH|BXE/BEE|
+-------+--------------------+-----+---+--------+-------+-------+
|   F661|KARBHARI SRUSHTI ...|   AB| AB|      AB|     AB|     AB|
|   F835|THORAT VISHAL ANKUSH|   AB| AB|      AB|     AB|     AB|
|   F843|OVHAL PRASHIK PRE...|   AB| AB|      AB|     AB|     AB|
|   F853|SHILVANT SAHIL SA...|   AB| AB|      AB|     AB|     AB|
|   F859|  DAUND PIYUSH ASHOK|   AB| AB|      AB|     AB|     AB|
+-------+--------------------+-----+---+--------+-------+-------+



In [8]:
print("Fail Student in one or more subject : ")
fail = df.filter(
    (col("EM-II") < 28 ) |
    (col("PPS") < 28 ) |
    (col("CHEM/PHY") < 28 ) |
    (col("EG/MECH") < 28 ) |
    (col("BXE/BEE") < 28 )
)
fail.show()

Fail Student in one or more subject : 
+-------+--------------------+-----+---+--------+-------+-------+
|Roll No|      Candidate Name|EM-II|PPS|CHEM/PHY|EG/MECH|BXE/BEE|
+-------+--------------------+-----+---+--------+-------+-------+
|   F101| RAI ADITYA RAJKUMAR|   31| 34|      26|     42|     22|
|   F102|DHOKCHAULE SHREYA...|   35| 39|      25|     54|     21|
|   F103|  GORE MANOJ TUKARAM|   46| 45|      26|     58|     19|
|   F104|GAVALI ROHIT PUNJ...|   19| 18|      18|     39|      7|
|   F105| GAIKWAD AMIT ANKUSH|   22| 24|      29|     52|     18|
|   F106|  JADHAV SAYALI BAPU|   39| 37|      43|     44|     27|
|   F107|VINCHURKAR PARTH ...|   AB| AB|      29|     19|      5|
|   F108|NAGPURE AKSHATA N...|   28| 29|      43|     46|     21|
|   F109|PATTEWAR SAMIKSHA...|   35| 37|      39|     29|     13|
|   F110|PANDHE SHRAVAN SA...|    9| 10|      12|     15|      0|
|   F111|JADHAO SARTHAK GA...|    9|  8|       7|     22|      1|
|   F112|PATHAN KASHAF NAA...|   31| 

In [9]:
print("Fail Student In All Subject : ")
fail = df.filter(
    (col("EM-II") < 28 ) &
    (col("PPS") < 28 ) &
    (col("CHEM/PHY") < 28 ) &
    (col("EG/MECH") < 28 ) &
    (col("BXE/BEE") < 28 )
)
fail.show()

Fail Student In All Subject : 
+-------+--------------------+-----+---+--------+-------+-------+
|Roll No|      Candidate Name|EM-II|PPS|CHEM/PHY|EG/MECH|BXE/BEE|
+-------+--------------------+-----+---+--------+-------+-------+
|   F110|PANDHE SHRAVAN SA...|    9| 10|      12|     15|      0|
|   F111|JADHAO SARTHAK GA...|    9|  8|       7|     22|      1|
|   F116|  BHUSHANVINODKODMUR|    9|  7|       0|      0|      0|
|   F117|LOKHANDE GANESH S...|    4|  5|       0|      0|      0|
|   F135|  MANG PRASHANT BABU|   10|  9|      26|      0|      5|
|   F136|THORBOLE DNYANESH...|    9| 10|      18|      4|      0|
|   F140|   SHELKE SURAJ RAMA|    4|  5|       0|      0|      0|
|   F141|KALE SHRAVANI NAV...|   12| 15|       0|     14|      8|
|   F145|BHOKNAL PRANAV PA...|   16| 17|       9|      4|      0|
|   F147|DUKALE PRUTHVIRAJ...|   10|  9|      10|      2|      0|
|   F151|  GADADE AMAR SANDIP|   13| 14|      13|      5|      2|
|   F153| UDHAN KRUSHNA BANDU|   22| 25|     

In [10]:
print(df.columns)

['Roll No', 'Candidate Name', 'EM-II', 'PPS', 'CHEM/PHY', 'EG/MECH', 'BXE/BEE']


In [11]:

subjects = ["EM-II", "PPS", "CHEM/PHY", "EG/MECH", "BXE/BEE"]

for sub in subjects:
    df = df.withColumn(sub, when(col(sub) == "AB", 0).otherwise(col(sub).cast("int")))


In [12]:
print("Total Marks & Percentage :")
df = df.withColumn("Total", reduce(operator.add, [col(sub) for sub in subjects]))
 

max_marks_per_subject = 70
total_max = max_marks_per_subject * len(subjects)

df = df.withColumn("Percentage", round((col("Total") / total_max) * 100, 2)) 

df.select("Roll No", "Candidate Name", "Total", "Percentage").show(10, truncate=False)

Total Marks & Percentage :
+-------+--------------------------+-----+----------+
|Roll No|Candidate Name            |Total|Percentage|
+-------+--------------------------+-----+----------+
|F101   |RAI ADITYA RAJKUMAR       |155  |44.29     |
|F102   |DHOKCHAULE SHREYA RAVINDRA|174  |49.71     |
|F103   |GORE MANOJ TUKARAM        |194  |55.43     |
|F104   |GAVALI ROHIT PUNJARAM     |101  |28.86     |
|F105   |GAIKWAD AMIT ANKUSH       |145  |41.43     |
|F106   |JADHAV SAYALI BAPU        |190  |54.29     |
|F107   |VINCHURKAR PARTH SANTOSH  |53   |15.14     |
|F108   |NAGPURE AKSHATA NITIN     |167  |47.71     |
|F109   |PATTEWAR SAMIKSHA BALAJI  |153  |43.71     |
|F110   |PANDHE SHRAVAN SACHIN     |46   |13.14     |
+-------+--------------------------+-----+----------+
only showing top 10 rows



In [13]:
df.show()

+-------+--------------------+-----+---+--------+-------+-------+-----+----------+
|Roll No|      Candidate Name|EM-II|PPS|CHEM/PHY|EG/MECH|BXE/BEE|Total|Percentage|
+-------+--------------------+-----+---+--------+-------+-------+-----+----------+
|   F101| RAI ADITYA RAJKUMAR|   31| 34|      26|     42|     22|  155|     44.29|
|   F102|DHOKCHAULE SHREYA...|   35| 39|      25|     54|     21|  174|     49.71|
|   F103|  GORE MANOJ TUKARAM|   46| 45|      26|     58|     19|  194|     55.43|
|   F104|GAVALI ROHIT PUNJ...|   19| 18|      18|     39|      7|  101|     28.86|
|   F105| GAIKWAD AMIT ANKUSH|   22| 24|      29|     52|     18|  145|     41.43|
|   F106|  JADHAV SAYALI BAPU|   39| 37|      43|     44|     27|  190|     54.29|
|   F107|VINCHURKAR PARTH ...|    0|  0|      29|     19|      5|   53|     15.14|
|   F108|NAGPURE AKSHATA N...|   28| 29|      43|     46|     21|  167|     47.71|
|   F109|PATTEWAR SAMIKSHA...|   35| 37|      39|     29|     13|  153|     43.71|
|   

In [14]:
topper = df.orderBy(col("Percentage").desc()).limit(1)

print("Topper :")
topper.show()


Topper :
+-------+--------------------+-----+---+--------+-------+-------+-----+----------+
|Roll No|      Candidate Name|EM-II|PPS|CHEM/PHY|EG/MECH|BXE/BEE|Total|Percentage|
+-------+--------------------+-----+---+--------+-------+-------+-----+----------+
|   F610|BHALEKAR BHAKTI B...|   63| 46|      67|     68|     64|  308|      88.0|
+-------+--------------------+-----+---+--------+-------+-------+-----+----------+



In [15]:
stu = df.filter(col("Roll No") == "F601").show()

+-------+------------------+-----+---+--------+-------+-------+-----+----------+
|Roll No|    Candidate Name|EM-II|PPS|CHEM/PHY|EG/MECH|BXE/BEE|Total|Percentage|
+-------+------------------+-----+---+--------+-------+-------+-----+----------+
|   F601|AGLAVE SURAJ SHAHU|   38| 38|      43|     52|     42|  213|     60.86|
+-------+------------------+-----+---+--------+-------+-------+-----+----------+

